In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset_df = pd.read_csv("/content/dataset.csv")

In [3]:
dataset_df.head(10)

,Unnamed: 0,harmful,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation
5,5,0,[2/2] huge fan fare and big talking before the...
6,6,0,@user camping tomorrow @user @user @user @use...
7,7,0,the next school year is the year for exams.ð...
8,8,0,we won!!! love the land!!! #allin #cavs #champ...
9,9,0,@user @user welcome here ! i'm it's so #gr...


In [4]:
dataset_df.drop(columns=["Unnamed: 0"])

,harmful,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
56740,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
56741,1,"you've gone and broke the wrong heart baby, an..."
56742,1,young buck wanna eat!!.. dat nigguh like I ain...
56743,1,youu got wild bitches tellin you lies


In [5]:
dataset_df.shape

(56745, 3)

In [6]:
dataset_df['harmful'].value_counts()

,count
harmful,
0,32592
1,24153


In [7]:
for _ in range(10):
  random_index = np.random.randint(0, len(dataset_df))
  random_data = dataset_df.iloc[random_index]
  print(random_data["tweet"], random_data["harmful"])

enjoy being a kid. #summer #slides    0
this makes me   to answer your question @user - all of you &amp; all of these momentsâ¦  0
RT @CaLvii: @troncat1003 theyre afraid of being a bitch in person 1
RT @ajjacobs: Just learned that there was a 1932 Japanese political plot to assassinate Charlie Chaplin. #PleaseOptionThisTweetNow #Back ... 0
 @user i see where hillary got her inspiration for her $12,000 jacket. ð¤ð    0
thunderball - thunderdance (1994)  #cd   hardcore #oldschool techno #vk download here:   0
  flag day!  the chinese flag is full of deeper meaning. the red background represents the communist...  0
"LMFAOOOOOOOOOOOOOOOOOO @bb_brooks: If your pussy is good but my mama don&#8217;t like you? Guess who gotta go&#8230;&#8230;&#8230;..my mama." 1
Why was Jorge Zimmerman arrested? Do prosecutors have a witness that saw Zimmy attack TrayTray? Are they afraid of the hoodie pussies? 1
4 days until @user where nothing else matter except the music and having a good time!   #cam

Preprocessing

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
x = dataset_df["tweet"].values
y = dataset_df["harmful"].values

# The .values attribute converts the pandas Series (each column in a DataFrame) into a NumPy array. This is useful when you need to perform operations that require NumPy arrays or when you need to pass the data into machine learning models or other libraries that expect NumPy arrays as input.

In [10]:
print(x)

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"
 '  bihday your majesty' ...
 'young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again'
 'youu got wild bitches tellin you lies'
 '~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB']


In [11]:
sequences = [sequence for sequence in x]
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

In [12]:
type(sequences)

list

In [13]:
type(x)

numpy.ndarray

In [14]:
type(model_inputs)

transformers.tokenization_utils_base.BatchEncoding

In [15]:
import tensorflow as tf

In [16]:
dataset = tf.data.Dataset.from_tensor_slices((model_inputs['input_ids'], y))

In [17]:
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [18]:
# Training pipeline

train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional

model = Sequential(name="tweet-classifier")
model.add(Embedding(len(tokenizer.get_vocab()), 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [20]:
len(tokenizer.get_vocab())

30522

In [21]:
len(tokenizer)

30522

In [22]:
model.summary()

Model: "tweet-classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [32]:
# Using GPUs

with tf.device("/device:GPU:0"):
  history = model.fit(train, epochs=1, validation_data=val)

2482/2482 ━━━━━━━━━━━━━━━━━━━━ 72s 29ms/step - loss: 0.0025 - val_loss: 6.5660e-04


In [25]:
# # Using TPUs
# import tensorflow as tf

# # Detect and initialize the TPU
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)  # Create TPU strategy

# # Wrap your model training in the strategy scope
# with strategy.scope():
#     # model = ...  # Define your model here
#     # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     history = model.fit(train, epochs=1, validation_data=val)


In [45]:
# Tokenize the new text
text = "hail hitler"
new_input = tokenizer(text, padding=True, return_tensors="tf")

# Get the input_ids which is what your model was trained on
input_ids = new_input['input_ids']

# Make prediction
prediction = model.predict(input_ids)


# Print results
print(f"Raw prediction score: {prediction[0][0]}")
print(f"Binary prediction: {'Harmful' if prediction[0][0] > 0.5 else 'Not Harmful'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Raw prediction score: 1.0
Binary prediction: Harmful


In [36]:
prediction

array([[0.00010473]], dtype=float32)

In [28]:
# Evaluation metrics

from tensorflow.keras.metrics import Precision, Recall, Accuracy

pre = Precision()
rec = Recall()
acc = Accuracy()

import numpy as np

for batch in test.as_numpy_iterator():
    x_true, y_true = batch
    y_pred = model.predict(x_true)
    y_pred_labels = np.argmax(y_pred, axis=-1)  # Assuming multi-class classification

    pre.update_state(y_true, y_pred)
    rec.update_state(y_true, y_pred)
    acc.update_state(y_true, y_pred_labels)  # Ensure labels are used for accuracy



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━

In [29]:
precision = pre.result().numpy()
recall = rec.result().numpy()
accuracy = acc.result().numpy()

# Calculate F1 Score
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Precision: 0.9996
Recall: 0.9996
Accuracy: 0.5768
F1 Score: 0.9996


In [30]:
# Save with .keras extension (recommended for newer versions)
model.save('tweet_classifier_model.keras')

# OR save with .h5 extension (older format)
model.save_weights('tweet_classifier.weights.h5')

# BERT tokenizer
tokenizer.save_pretrained('bert_tokenizer')


('bert_tokenizer/tokenizer_config.json',
 'bert_tokenizer/special_tokens_map.json',
 'bert_tokenizer/vocab.txt',
 'bert_tokenizer/added_tokens.json')